In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix 

### Load subtype label, cancer label and predicted result of each patch.

In [2]:
#load patch list with corresponding labels - subtype label
test_set = pd.read_csv("/home5/hby/subtype_newdata/0.3/ALL/all_2000_test.txt", header=None, sep=' ')
test_set.columns = ['filename', 'label','subtypes']

In [4]:
# load corresponding predicted results
predict_all = pd.read_csv("/home5/hby/subtype_newdata/0.3/ssl_2/crd.csv", header=None)
predict_all.columns = ['cancer','normal','ccrcc', 'prcc', 'chrcc']
predict_all.reset_index(drop=True, inplace=True)

In [5]:
# check the patch list length is same or not with predicted list
print(predict_all.shape)
print(test_set.shape)

(91321, 5)
(91321, 3)


In [6]:
# concat two list
test_set = pd.concat([test_set, predict_all], axis=1)
test_set['slidename'] = test_set['filename'].apply(lambda x: x.split('/')[-2])
print(test_set.head(5))

                                            filename  label  subtypes  cancer  \
0  /home4/gzy/CancerRegionDetection/test/2000/can...      0         0  1.0872   
1  /home4/gzy/CancerRegionDetection/test/2000/can...      0         0  1.4979   
2  /home4/gzy/CancerRegionDetection/test/2000/can...      0         0  1.6272   
3  /home4/gzy/CancerRegionDetection/test/2000/can...      0         0  1.4342   
4  /home4/gzy/CancerRegionDetection/test/2000/can...      0         0  1.5876   

   normal   ccrcc    prcc   chrcc  \
0 -1.0884  2.4703 -1.4490 -2.3961   
1 -1.4982  2.5088 -1.4712 -2.4491   
2 -1.6281  2.6005 -1.4302 -3.4732   
3 -1.4343  2.7686 -1.5527 -2.8984   
4 -1.5876  2.6067 -1.6857 -2.3079   

                                           slidename  
0  TCGA-BP-4807-01Z-00-DX1_c985d6a4-5104-4b37-b7a...  
1  TCGA-BP-4807-01Z-00-DX1_c985d6a4-5104-4b37-b7a...  
2  TCGA-BP-4807-01Z-00-DX1_c985d6a4-5104-4b37-b7a...  
3  TCGA-BP-4807-01Z-00-DX1_c985d6a4-5104-4b37-b7a...  
4  TCGA-BP-4807

In [7]:
# softmax
predict_crd = np.array(predict_all[['cancer','normal']])
predict_crd_norm = np.zeros_like(predict_crd)
for i in range(predict_crd.shape[0]):
    predict_crd_norm[i,:] = np.exp(predict_crd[i,:])/sum(np.exp(predict_crd[i,:]))
test_set['crd'] = np.argmax(predict_crd, axis=1)
test_set[['cancer','normal']] = predict_crd_norm

### Calculate the overall AUROC value and AUROC values of each subtype

In [10]:
# calculate metric of crd task, 
from sklearn.metrics import classification_report, roc_curve, auc
targets = ['ccrcc','prcc','chrcc','overall']
for i in range(len(targets)):
    if i == 3:
        test_set_sub = test_set[test_set['subtypes'] != i]
    else:
        test_set_sub = test_set[test_set['subtypes'] == i]
    labels = test_set_sub['label'].apply(lambda x: 1-x).tolist()
    labels_pre = test_set_sub['cancer'].tolist()
    false_positive_rate1,true_positive_rate1,thresholds=roc_curve(labels, labels_pre)
    roc_auc1=auc(false_positive_rate1, true_positive_rate1)
    print("AUROC value of %s: %f"%(targets[i], roc_auc1))

AUROC value of ccrcc: 0.944641
AUROC value of prcc: 0.985973
AUROC value of chrcc: 0.964600
AUROC value of overall: 0.966246
